In [1]:
import os
import joblib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

# functions

In [2]:
def compute_RMSE(diff):
    if len(diff)>0:
        return np.sqrt(np.mean(diff.values**2))
        
def compute_MAE(diff):
    if len(diff)>0:
        return np.mean(np.abs(diff.values))

def plot_errors(train_data,test_data):
    test_diff = test_data['outofsample_error']
    train_diff = train_data['insample_error']
    
    test_plot = test_data[['date','spot_price','rho','v0']].copy()
    test_plot = test_plot.reset_index().set_index('date')
    test_plot['MAE'] = test_diff.resample('D').apply(compute_RMSE)
    test_plot['RMSE'] = test_diff.resample('D').apply(compute_RMSE)
    test_plot = test_plot.reset_index()
    test_plot = test_plot.drop_duplicates(subset=['date'],keep='last').set_index('date').drop(columns='calculation_date')
    
    train_plot = train_data[['date','spot_price','rho','v0']].copy()
    train_plot = train_plot.reset_index().set_index('date')
    train_plot['MAE'] = train_diff.resample('D').apply(compute_RMSE)
    train_plot['RMSE'] = train_diff.resample('D').apply(compute_RMSE)
    train_plot = train_plot.reset_index()
    train_plot = train_plot.drop_duplicates(subset=['date'],keep='last').set_index('date').drop(columns='calculation_date')
    trainx = pd.date_range(start=min(train_data.index),end=max(train_data.index),periods=train_plot.shape[0])
    train_plot.index = trainx

    
    testx = pd.date_range(start=min(test_data.index),end=max(test_data.index),periods=test_plot.shape[0])
    test_plot.index = testx
    
    
    fig,axs = plt.subplots(max(len(train_plot.columns),len(test_plot.columns)),figsize=(10,10),sharex=True)
    for i,col in enumerate(train_plot.columns):
        axs[i].plot(train_plot[col],color='green',label='in-sample')
        axs[i].set_title(col.replace('_',' '))
        axs[i].legend()
    for i,col in enumerate(test_plot.columns):
        axs[i].plot(test_plot[col],color='purple',label='out-of-sample')
        axs[i].set_title(col.replace('_',' '))
        axs[i].legend()
    plt.show()

# loading model

In [3]:
from model_settings import ms
root = Path().resolve().parent.parent
models_dir = os.path.join(root,ms.trained_models)
models = pd.Series([f for f in os.listdir(models_dir) if f.find('Legacy')==-1])
for i,m in enumerate(models):
    print(f"{i}     {m}")

0     2024-11-09 024203778497 cboe spx relative barrier
1     2024-11-09 112825652640 cboe spx relative asian


In [4]:
selected_model = models.iloc[1]
model_dir = os.path.join(models_dir,selected_model)
pickle = [f for f in os.listdir(model_dir) if f.endswith('.pkl')][0]
picke_dir = os.path.join(model_dir,pickle)
model = joblib.load(picke_dir)
model

{'seed': 1312,
 'raw_data':         spot_price  strike_price  days_to_maturity  n_fixings  \
 0           1274.0        637.00                 7        1.0   
 1           1274.0        637.00                 7        1.0   
 2           1274.0        637.00                 7        1.0   
 3           1274.0        637.00                 7        1.0   
 4           1274.0        955.50                 7        1.0   
 ...            ...           ...               ...        ...   
 338935      5857.0       7320.75                84        1.0   
 338936      5857.0       8785.00                84        1.0   
 338937      5857.0       8785.00                84        1.0   
 338938      5857.0       8785.00                84        1.0   
 338939      5857.0       8785.00                84        1.0   
 
         fixing_frequency  past_fixings averaging_type     w  risk_free_rate  \
 0                      7             0      geometric  call        0.001578   
 1                 

In [5]:
print('model attributes:\n')
for k in model.keys():
    print(k)

model attributes:

seed
raw_data
dataset
target_name
excluded_features
numerical_features
categorical_features
feature_set
n_features
development_dates
test_dates
train_data
test_data
train_X
train_y
test_X
test_y
preprocessor
pipeline
model
model_fit
dnn_runtime
numerical_scaler
dnn_params
transformers
regressor
dnn_pipeline


In [6]:
for col in model['feature_set']:
    print(f"{col.replace("_"," ")}:",f"\n{model['test_data'][col].copy().squeeze().sort_values().drop_duplicates().reset_index(drop=True)}\n")
print()

days to maturity: 
0     7
1    28
2    84
Name: days_to_maturity, dtype: int64

fixing frequency: 
0     7
1    28
2    84
Name: fixing_frequency, dtype: int64

past fixings: 
0    0
Name: past_fixings, dtype: int64

risk free rate: 
0      0.000330
1      0.000355
2      0.000380
3      0.000406
4      0.000431
         ...   
781    0.054494
782    0.054500
783    0.054513
784    0.054801
785    0.054889
Name: risk_free_rate, Length: 786, dtype: float64

dividend rate: 
0       0.012624
1       0.012628
2       0.012641
3       0.012671
4       0.012687
          ...   
1152    0.023672
1153    0.023681
1154    0.023780
1155    0.023823
1156    0.024133
Name: dividend_rate, Length: 1157, dtype: float64

kappa: 
0       3.553361e-10
1       3.924365e-10
2       7.518568e-10
3       8.464905e-10
4       1.055398e-09
            ...     
1747    6.161121e+01
1748    6.214923e+01
1749    6.690466e+01
1750    8.814194e+01
1751    1.158504e+02
Name: kappa, Length: 1752, dtype: float64

th

In [7]:
train_data = model['train_data'].copy()
test_data = model['test_data'].copy()
train_data['calculation_date'] = pd.to_datetime(train_data['calculation_date'],format='mixed')
test_data['calculation_date'] = pd.to_datetime(test_data['calculation_date'],format='mixed')
test_data = test_data.set_index('calculation_date').sort_index()
train_data = train_data.set_index('calculation_date').sort_index()
print(model['feature_set'])

['days_to_maturity', 'fixing_frequency', 'past_fixings', 'risk_free_rate', 'dividend_rate', 'kappa', 'theta', 'rho', 'eta', 'v0', 'relative_spot', 'averaging_type', 'w']


# retraining

In [8]:
from convsklearn import convsklearn
help(convsklearn)

Help on class convsklearn in module convsklearn.convsklearn:

class convsklearn(builtins.object)
 |  convsklearn(target_name='observed_price', excluded_features=['barrier_price', 'asian_price', 'observed_price', 'outin', 'updown', 'n_fixings'], seed=1312)
 |
 |  a proprietary class of convenience wrappers for sklearn
 |
 |  Methods defined here:
 |
 |  __init__(self, target_name='observed_price', excluded_features=['barrier_price', 'asian_price', 'observed_price', 'outin', 'updown', 'n_fixings'], seed=1312)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  load_data(self, data)
 |
 |  preprocess_data(self, development_dates, test_dates, plot=True)
 |
 |  run_dnn(self, print_details=True)
 |
 |  test_prediction_accuracy(self)
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



In [9]:
test_dates = model['test_dates']
full_dataset = model['dataset']
all_dates = full_dataset['date'].drop_duplicates().sort_values().reset_index(drop=True)
all_dates

0      2012-01-03
1      2012-01-05
2      2012-01-06
3      2012-01-10
4      2012-01-18
          ...    
1409   2024-09-24
1410   2024-10-07
1411   2024-10-08
1412   2024-10-09
1413   2024-10-14
Name: date, Length: 1414, dtype: datetime64[ns]

In [ ]:
retraining_frequency = 30
retraining_dates = test_dates.iloc[0::retraining_frequency]
retraining_dates = retraining_dates.reset_index(drop=True)

cols = ['cpu','isMAE','isRMSE','osMAE','osRMSE']
df = pd.DataFrame(np.tile(np.nan,(len(retraining_dates),len(cols))),columns=cols,index=retraining_dates)

models = {}
bar = tqdm(total=len(retraining_dates),leave=True)
for date in retraining_dates:
    print(date)
    development_dates = all_dates[all_dates<=date]
    subset_test_dates = all_dates[~all_dates.isin(development_dates)].iloc[:retraining_frequency]
    retrainer = convsklearn()
    retrainer.excluded_features += model['excluded_features']
    retrainer.target_name = model['target_name']
    retrainer.load_data(full_dataset)
    print(f"training:  {min(development_dates).strftime('%A, %d %B, %Y')}  -  {max(development_dates).strftime('%A, %d %B, %Y')} ({len(development_dates)} trading days)")
    print(f"testing:   {min(subset_test_dates).strftime('%A, %d %B, %Y')}  -  {max(subset_test_dates).strftime('%A, %d %B, %Y')} ({len(subset_test_dates)} trading days)")
    retrainer.preprocess_data(development_dates, subset_test_dates,plot=False)
    retrainer.run_dnn(print_details=False)
    retrainer.test_prediction_accuracy()
    models[date] = retrainer.__dict__
    m = retrainer.__dict__
    df.at[date,'cpu'] = m['dnn_runtime']
    df.at[date,'isMAE'] = compute_MAE(m['train_data']['insample_error'])
    df.at[date,'isRMSE'] = compute_RMSE(m['train_data']['insample_error'])
    df.at[date,'osMAE'] = compute_MAE(m['test_data']['outofsample_error'])
    df.at[date,'osRMSE'] = compute_RMSE(m['test_data']['outofsample_error'])
    print(df.dropna())
    print()
    bar.update(1)
bar.close()

  0%|                                                                                                                                                                                     | 0/44 [00:00<?, ?it/s]

2012-11-06 00:00:00
training:  Tuesday, 03 January, 2012  -  Tuesday, 06 November, 2012 (101 trading days)
testing:   Thursday, 08 November, 2012  -  Friday, 22 March, 2013 (30 trading days)


  2%|███▉                                                                                                                                                                         | 1/44 [00:02<01:31,  2.13s/it]


in sample:
     RMSE: 0.004479471496554727
     MAE: 0.002656542823136828

out of sample:
     RMSE: 0.01002161364404371
     MAE: 0.005195466698951306
                 cpu     isMAE    isRMSE     osMAE    osRMSE
date                                                        
2012-11-06  2.036912  0.002657  0.004479  0.005195  0.010022

2013-03-22 00:00:00
training:  Tuesday, 03 January, 2012  -  Friday, 22 March, 2013 (131 trading days)
testing:   Tuesday, 26 March, 2013  -  Monday, 10 June, 2013 (30 trading days)


  5%|███████▊                                                                                                                                                                     | 2/44 [00:07<02:42,  3.87s/it]

...


in sample:
     RMSE: 0.003944732331294065
     MAE: 0.002175192029035582

out of sample:
     RMSE: 0.004052503173446884
     MAE: 0.002191926418961727
                  cpu     isMAE    isRMSE     osMAE    osRMSE
date                                                         
2012-11-06   2.036912  0.002657  0.004479  0.005195  0.010022
2013-03-22   4.950001  0.002976  0.004916  0.002807  0.004670
2013-06-10   5.825000  0.002881  0.004759  0.002677  0.004503
2013-08-29   7.242001  0.002432  0.004262  0.002329  0.004458
2013-11-18   8.051924  0.002765  0.004583  0.002763  0.004445
2014-02-14   9.171103  0.002556  0.004379  0.002282  0.003985
2014-06-04  10.096912  0.002988  0.004799  0.002675  0.004319
2014-08-13  11.832237  0.002383  0.004195  0.002200  0.003907
2014-12-01  12.156193  0.002362  0.004153  0.003267  0.006201
2015-04-17  13.433484  0.002535  0.004329  0.002374  0.004046
2015-07-20  14.306873  0.002184  0.003977  0.002449  0.004025
2015-11-17  15.257175  0.002620  0.00441

 36%|██████████████████████████████████████████████████████████████▌                                                                                                             | 16/44 [03:10<08:13, 17.61s/it]


in sample:
     RMSE: 0.004246748784352126
     MAE: 0.002472581894306519

out of sample:
     RMSE: 0.004187294156966989
     MAE: 0.002301137833496158
                  cpu     isMAE    isRMSE     osMAE    osRMSE
date                                                         
2012-11-06   2.036912  0.002657  0.004479  0.005195  0.010022
2013-03-22   4.950001  0.002976  0.004916  0.002807  0.004670
2013-06-10   5.825000  0.002881  0.004759  0.002677  0.004503
2013-08-29   7.242001  0.002432  0.004262  0.002329  0.004458
2013-11-18   8.051924  0.002765  0.004583  0.002763  0.004445
2014-02-14   9.171103  0.002556  0.004379  0.002282  0.003985
2014-06-04  10.096912  0.002988  0.004799  0.002675  0.004319
2014-08-13  11.832237  0.002383  0.004195  0.002200  0.003907
2014-12-01  12.156193  0.002362  0.004153  0.003267  0.006201
2015-04-17  13.433484  0.002535  0.004329  0.002374  0.004046
2015-07-20  14.306873  0.002184  0.003977  0.002449  0.004025
2015-11-17  15.257175  0.002620  0.00441

 39%|██████████████████████████████████████████████████████████████████▍                                                                                                         | 17/44 [03:31<08:25, 18.73s/it]


in sample:
     RMSE: 0.0039662605872879085
     MAE: 0.002207184511228894

out of sample:
     RMSE: 0.00402631972550385
     MAE: 0.002120281137277737
                  cpu     isMAE    isRMSE     osMAE    osRMSE
date                                                         
2012-11-06   2.036912  0.002657  0.004479  0.005195  0.010022
2013-03-22   4.950001  0.002976  0.004916  0.002807  0.004670
2013-06-10   5.825000  0.002881  0.004759  0.002677  0.004503
2013-08-29   7.242001  0.002432  0.004262  0.002329  0.004458
2013-11-18   8.051924  0.002765  0.004583  0.002763  0.004445
2014-02-14   9.171103  0.002556  0.004379  0.002282  0.003985
2014-06-04  10.096912  0.002988  0.004799  0.002675  0.004319
2014-08-13  11.832237  0.002383  0.004195  0.002200  0.003907
2014-12-01  12.156193  0.002362  0.004153  0.003267  0.006201
2015-04-17  13.433484  0.002535  0.004329  0.002374  0.004046
2015-07-20  14.306873  0.002184  0.003977  0.002449  0.004025
2015-11-17  15.257175  0.002620  0.00441

In [ ]:
from plotters import PlotCols
PlotCols(df)

In [ ]:
retrained = {'errors':df,'models':models}
tag = ms.timetag()
joblib.dump(retraiend,f'{tag}retrained_relative_asian_options.pkl')